In [1]:
cd ..

/conv


In [2]:
%autosave 600
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
#%load_ext autoreload
#%autoreload 2
#%pdb

import os
#os.environ['CUDA_VISIBLE_DEVICES'] = ""

import sys
sys.path.append("notebooks")

import numpy as np
import matplotlib.pyplot as plt
import torch
torch.set_num_threads(8)

Autosaving every 600 seconds


In [3]:
import numpy as np
import pdb
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

from utils.data import get_train_dev_test_ssl
from pretrained import load_pretrained
from skssl.classifiers import LabelSpreading
from skssl.meta import SelfTrainingMeta
from skssl.classifiers.sslvae import SSLVAE, SSLVAELoss
from skssl.training import NeuralNetClassifier, NeuralNetEstimator
from skssl.predefined import MLP

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Supervised All Labels

In [4]:
"""
for dataset in ["mnist","svhn","cifar10"]:
    print(dataset)
    train, dev, test = get_train_dev_test_ssl(dataset)
    
    classifier = load_pretrained("supervised", dataset, is_all_labels=True, is_retrain=False)
    
    print(dataset, classifier.score(test, test.targets))
"""
    
print("mnist 0.9933")
print("svhn 0.9596266133988937")
print("cifar10 0.8974")

mnist 0.9933
svhn 0.9596266133988937
cifar10 0.8974


# GNP ALL Labels

In [11]:
print("mnist 0.9953")
print("svhn 0.958090")
print("cifar10 0.8421")

mnist 0.9953
svhn 0.958090
cifar10 0.8421


# Supervised

In [5]:
"""
import random
for run in range(2):
    for dataset in ["mnist","svhn","cifar10"]:
        train, dev, test = get_train_dev_test_ssl(dataset)

        classifier = load_pretrained("supervised", dataset, is_retrain=True, suffix="_run{}".format(run), 
                                     seed=random.randint(1000))

        print(dataset, classifier.score(test, test.targets))
"""

'\nimport random\nfor run in range(2):\n    for dataset in ["mnist","svhn","cifar10"]:\n        train, dev, test = get_train_dev_test_ssl(dataset)\n\n        classifier = load_pretrained("supervised", dataset, is_retrain=True, suffix="_run{}".format(run), \n                                     seed=random.randint(1000))\n\n        print(dataset, classifier.score(test, test.targets))\n'

In [6]:
"""
for run in range(3):
    for dataset in ["mnist","svhn","cifar10"]:
        train, dev, test = get_train_dev_test_ssl(dataset)

        classifier = load_pretrained("supervised", dataset, is_retrain=False ,
                                     suffix="_run{}".format(run) if run != 2 else "")

        print(dataset, classifier.score(test, test.targets))
"""

'\nfor run in range(3):\n    for dataset in ["mnist","svhn","cifar10"]:\n        train, dev, test = get_train_dev_test_ssl(dataset)\n\n        classifier = load_pretrained("supervised", dataset, is_retrain=False ,\n                                     suffix="_run{}".format(run) if run != 2 else "")\n\n        print(dataset, classifier.score(test, test.targets))\n'

In [7]:
print("mnist", np.array([0.6613, 0.6232, 0.7111]).mean(), np.array([0.6613, 0.6232, 0.7111]).std())
print("cifar10", np.array([0.7105, 0.6635, 0.7272]).mean(), np.array([0.7105, 0.6635, 0.7272]).std())
print("svhn", np.array([0.8305931161647203, 0.8355869698832207, 0.8481100184388445]).mean(), 
      np.array([0.8305931161647203, 0.8355869698832207, 0.8481100184388445]).std())

mnist 0.6652 0.035990832165983595
cifar10 0.7004 0.02696825294057193
svhn 0.8380967014955951 0.007368153585755297


# M1

### All Labes

In [6]:
# could use pipleine but want only to transform once
transformed = dict() # store all
scores = dict()
is_all_labels = True

for dataset in ["mnist","svhn"]: #"cifar10"
    transformed[dataset] = dict()
    train, dev, test = get_train_dev_test_ssl(dataset, is_all_labels=is_all_labels)
    
    for enc_dec in ["resnet"]: # "cnn" , "resnetEnc_cnnDec"  much worst
        vae = load_pretrained("vae", dataset, enc_dec=enc_dec).freeze()
        transformed[dataset][(enc_dec,"train")] = vae.transform(train)
        transformed[dataset][(enc_dec,"test")] = vae.transform(test)
        
        print()
        for name, classifier in [("LabelSpreading",LabelSpreading()),
                                 #("SelfTraining MLP", SelfTrainingMeta(MLPClassifier(solver="lbfgs"), max_iter=50)), # much worst
                                 #("Supervised MLP", MLPClassifier(solver="lbfgs"))
                                ]:
            
            _ = classifier.fit(transformed[dataset][(enc_dec,"train")], train.targets)
            y_hat = classifier.predict(transformed[dataset][(enc_dec,"test")])
            scores[(dataset, enc_dec, name)] = accuracy_score(test.targets, y_hat)
            print((dataset, enc_dec, name), scores[(dataset, enc_dec, name)])
            




('mnist', 'resnet', 'LabelSpreading') 0.9281
Using downloaded and verified file: /conv/utils/data/../../data/SVHN/train_32x32.mat
Using downloaded and verified file: /conv/utils/data/../../data/SVHN/test_32x32.mat

('svhn', 'resnet', 'LabelSpreading') 0.5156730178242164


### SSL

In [4]:


# could use pipleine but want only to transform once
transformed = dict() # store all
scores = dict()
is_all_labels = False

for dataset in ["mnist","svhn"]: #"cifar10"
    transformed[dataset] = dict()
    train, dev, test = get_train_dev_test_ssl(dataset, is_all_labels=is_all_labels)
    
    for enc_dec in ["resnet"]: # "cnn" , "resnetEnc_cnnDec"  much worst
        vae = load_pretrained("vae", dataset, enc_dec=enc_dec).freeze()
        transformed[dataset][(enc_dec,"train")] = vae.transform(train)
        transformed[dataset][(enc_dec,"test")] = vae.transform(test)
        
        print()
        for name, classifier in [("LabelSpreading",LabelSpreading()),
                                 #("SelfTraining MLP", SelfTrainingMeta(MLPClassifier(solver="lbfgs"), max_iter=50)), # much worst
                                 #("Supervised MLP", MLPClassifier(solver="lbfgs"))
                                ]:
            
            _ = classifier.fit(transformed[dataset][(enc_dec,"train")], train.targets)
            y_hat = classifier.predict(transformed[dataset][(enc_dec,"test")])
            scores[(dataset, enc_dec, name)] = accuracy_score(test.targets, y_hat)
            print((dataset, enc_dec, name), scores[(dataset, enc_dec, name)])
            
# ('mnist', 'resnetEnc_cnnDec', 'LabelSpreading') 0.2797
# ('mnist', 'resnetEnc_cnnDec', 'SelfTraining MLP') 0.1135
# ('mnist', 'resnet', 'LabelSpreading') 0.8554
# ('mnist', 'resnet', 'SelfTraining MLP') 0.8045
# ('mnist', 'cnn', 'LabelSpreading') 0.1032
# ('mnist', 'cnn', 'SelfTraining MLP') 0.1135



KeyboardInterrupt



# SSLVAE

## M2

In [4]:
for dataset in ["mnist","svhn","cifar10"]:
    train, dev, test = get_train_dev_test_ssl(dataset)
    
    classifier = load_pretrained("sslvae", dataset, mode="m2", z_dim=64)
    
    print(dataset, classifier.score(test, test.targets))

/master/skssl/predefined/mlp.py:52: UserWarning: hidden_size=32 smaller than output=128 and input=74. Setting it to 74.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))


mnist 0.9654
Using downloaded and verified file: /master/utils/data/../../data/SVHN/train_32x32.mat
Using downloaded and verified file: /master/utils/data/../../data/SVHN/test_32x32.mat


/master/skssl/predefined/mlp.py:52: UserWarning: hidden_size=32 smaller than output=128 and input=74. Setting it to 74.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))


svhn 0.15964966195451752
Files already downloaded and verified
Files already downloaded and verified


/master/skssl/predefined/mlp.py:52: UserWarning: hidden_size=32 smaller than output=128 and input=74. Setting it to 74.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))


cifar10 0.4695


## M1 + M2

In [5]:
for dataset in ["mnist","svhn","cifar10"]:
    train, dev, test = get_train_dev_test_ssl(dataset)
    
    classifier = load_pretrained("sslvae", dataset, mode="m1+m2", z_dim=64)
    
    print(dataset, classifier.score(test, test.targets))

/master/skssl/predefined/mlp.py:52: UserWarning: hidden_size=32 smaller than output=64 and input=64. Setting it to 64.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))
/master/skssl/predefined/mlp.py:52: UserWarning: hidden_size=32 smaller than output=128 and input=74. Setting it to 74.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))
/master/skssl/predefined/mlp.py:52: UserWarning: hidden_size=32 smaller than output=64 and input=74. Setting it to 64.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))


mnist 0.4478
Using downloaded and verified file: /master/utils/data/../../data/SVHN/train_32x32.mat
Using downloaded and verified file: /master/utils/data/../../data/SVHN/test_32x32.mat


/master/skssl/predefined/mlp.py:52: UserWarning: hidden_size=32 smaller than output=64 and input=64. Setting it to 64.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))
/master/skssl/predefined/mlp.py:52: UserWarning: hidden_size=32 smaller than output=128 and input=74. Setting it to 74.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))
/master/skssl/predefined/mlp.py:52: UserWarning: hidden_size=32 smaller than output=64 and input=74. Setting it to 64.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))


svhn 0.301359864781807
Files already downloaded and verified
Files already downloaded and verified


/master/skssl/predefined/mlp.py:52: UserWarning: hidden_size=32 smaller than output=64 and input=64. Setting it to 64.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))
/master/skssl/predefined/mlp.py:52: UserWarning: hidden_size=32 smaller than output=128 and input=74. Setting it to 74.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))
/master/skssl/predefined/mlp.py:52: UserWarning: hidden_size=32 smaller than output=64 and input=74. Setting it to 64.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))


cifar10 0.2093


## Auxiliary

In [6]:
for dataset in ["mnist","svhn","cifar10"]:
    train, dev, test = get_train_dev_test_ssl(dataset)
    
    classifier = load_pretrained("sslvae", dataset, mode="auxiliary", z_dim=64)
    
    print(dataset, classifier.score(test, test.targets))

/master/skssl/predefined/mlp.py:52: UserWarning: hidden_size=32 smaller than output=128 and input=64. Setting it to 64.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))
/master/skssl/predefined/mlp.py:52: UserWarning: hidden_size=32 smaller than output=128 and input=138. Setting it to 128.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))


FileNotFoundError: [Errno 2] No such file or directory: 'results/pretrained/sslvae/auxiliary_z64_mnist/history.json'